In [4]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_train,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------

image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

import boto3

steps = execution.list_steps()
sm = boto3.client("sagemaker")

training_job_arn = None

for step in steps:
    if step["StepName"] == "TrainingStep":
        training_job_arn = step["Metadata"]["TrainingJob"]["Arn"]
        break

# Extract training job name from ARN
training_job_name = training_job_arn.split("/")[-1]
print("Training job:", training_job_name)
s3_model_path = f"s3://{bucket}/models/{training_job_name}/output/model.tar.gz"
print("s3_model_path:", s3_model_path)

        

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri_inference,
    model_data=s3_model_path,
    role=role,
    # entry_point="serve.py",       # ✅ Add this line
    # source_dir=".",               # ✅ Or directory containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint15",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # generate the report every 5 minutes
    schedule_cron_expression="cron(0/5 * * * ? *)"
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subn

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:155                                                                                  │
│                                                                                                  │
│   152 )                                                                                          │
│   153                                                                                            │
│   154 if predictor is None:                                                                      │
│ ❱ 155 │   raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")      │
│   156                                                                                            │
│   157 response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,2   │
│   158 print(response)                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 🚨 model.deploy() returned None — deployment likely failed.

In [6]:
if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 if predictor is None:                                                                        │
│ ❱ 2 │   raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")        │
│   3                                                                                              │
│   4 response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,2     │
│   5 print(response)                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 🚨 model.deploy() returned None — deployment likely failed.

In [ ]:

image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

import boto3

steps = execution.list_steps()
sm = boto3.client("sagemaker")

training_job_arn = None

for step in steps:
    if step["StepName"] == "TrainingStep":
        training_job_arn = step["Metadata"]["TrainingJob"]["Arn"]
        break

# Extract training job name from ARN
training_job_name = training_job_arn.split("/")[-1]
print("Training job:", training_job_name)
s3_model_path = f"s3://{bucket}/models/{training_job_name}/output/model.tar.gz"
print("s3_model_path:", s3_model_path)

        

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri_inference,
    model_data=s3_model_path,
    role=role,
    entry_point="serve.py",       # ✅ Add this line
    source_dir=".",               # ✅ Or directory containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint16",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



In [ ]:
import boto3


sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)

predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint19"
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



<h1>Pipeline where the inference uses the registry model instead of the trained model</h1> 

In [3]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"
image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    # source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_inference,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------



import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

steps = execution.list_steps()
sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)

predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint-20"
)

# If SDK returns None, manually create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

if predictor is None:
    # predictor = Predictor(endpoint_name="mlops-prod-endpoint-20", sagemaker_session=sagemaker_session)

    predictor = Predictor(
        endpoint_name="mlops-prod-endpoint-20",
        sagemaker_session=sagemaker_session,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
    )


response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # generate the report every 5 minutes
    schedule_cron_expression="cron(0/5 * * * ? *)"
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subn

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:138                                                                                  │
│                                                                                                  │
│   135 )                                                                                          │
│   136                                                                                            │
│   137 if predictor is None:                                                                      │
│ ❱ 138 │   raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")      │
│   139                                                                                            │
│   140 response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,2   │
│   141 print(response)                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 🚨 model.deploy() returned None — deployment likely failed.

In [9]:

import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

steps = execution.list_steps()
sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)

predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint-24"
)

# If SDK returns None, manually create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import StringDeserializer

if predictor is None:
    # predictor = Predictor(endpoint_name="mlops-prod-endpoint-20", sagemaker_session=sagemaker_session)

    predictor = Predictor(
        endpoint_name="mlops-prod-endpoint-24",
        sagemaker_session=sagemaker_session,
        serializer=CSVSerializer(),
        deserializer=StringDeserializer(),
    )


response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
------![False]


In [5]:
predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0") 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,     │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_predictor.py:213 in predict               │
│                                                                                                  │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│   212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│ ❱ 213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│   216 │   │   """Placeholder docstring"""                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_predictor.py:219 in _handle_response      │
│                                                                                                  │
│   216 │   │   """Placeholder docstring"""                                                        │
│   217 │   │   response_body = response["Body"]                                                   │
│   218 │   │   content_type = response.get("ContentType", "application/octet-stream")             │
│ ❱ 219 │   │   return self.deserializer.deserialize(response_body, content_type)                  │
│   220 │                                                                                          │
│   221 │   def _create_request_args(                                                              │
│   222 │   │   self,                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_deserializers.py:277 in deserialize       │
│                                                                                                  │
│   274 │   │   │   object: The JSON-formatted data deserialized into a Python object.             │
│   275 │   │   """                                                                                │
│   276 │   │   try:                                                                               │
│ ❱ 277 │   │   │   return json.load(codecs.getreader("utf-8")(stream))                            │
│   278 │   │   finally:                                                                           │
│   279 │   │   │   stream.close()                                                                 │
│   280                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/json/__init__.py:293 in load                                           │
│                                                                                                  │
│   290 │   To use a custom ``JSONDecoder`` subclass, specify it with the ``cls``                  │
│   291 │   kwarg; otherwise ``JSONDecoder`` is used.        

In [ ]:
execution.list_steps()


<h1>Check status of the endpoint via boto3</h1>
If you see:

- Status: Deleting → you must wait until deletion finishes.
- Status: Failed or anything else → force delete or use a new name.

In [ ]:
import boto3
sm = boto3.client('sagemaker')

endpoint_name = 'mlops-prod-endpoint3'

try:
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    print("Status:", response['EndpointStatus'])
except sm.exceptions.ClientError as e:
    if "Could not find" in str(e):
        print("✅ Endpoint does not exist")
    else:
        raise e
